# Mixtral in Colab

Welcome! In this notebook you can run [Mixtral8x7B-Instruct](https://huggingface.co/mistralai/Mixtral-8x7B-v0.1) with decent generation speed **right in Google Colab or on a consumer-grade GPU**. This was made possible by quantizing the original model in mixed precision and implementing a MoE-specific offloading strategy.

To learn more, read our [tech report](https://arxiv.org/abs/2312.17238) or check out the [repo](https://github.com/dvmazur/mixtral-offloading) on GitHub.

One will need approximately 16 GB of VRAM and 11 GB of RAM to run this notebook and generate somewhat long texts.


<details>

<summary>How to balance between RAM and GPU VRAM usage</summary>

You can balance between RAM and GPU VRAM usage by changing <code>offload_per_layer</code> variable in the <a href="#scrollTo=_mIpePTMFyRY&line=10&uniqifier=1">Initialize model</a> section. Increasing <code>offload_per_layer</code> will decrease GPU VRAM usage, increase RAM usage and decrease generation speed. Decreasing <code>offload_per_layer</code> will have the opposite effect.

Note that this notebook should run normally in Google Colab with <code>offload_per_layer = 4</code>, but may crush with other values. However, if you run this somewhere else, you're free to play with this variable.
</details>

## Install and import libraries

In [1]:
# fix numpy in colab
import numpy
from IPython.display import clear_output
import gc

# fix triton in colab
!export LC_ALL="en_US.UTF-8"
!export LD_LIBRARY_PATH="/usr/lib64-nvidia"
!export LIBRARY_PATH="/usr/local/cuda/lib64/stubs"
!ldconfig /usr/lib64-nvidia

# !git clone https://github.com/type-shangshu/mixtral-offloading.git --quiet
# !cd mixtral-offloading && pip install -q -r requirements.txt
# !huggingface-cli download lavawolfiee/Mixtral-8x7B-Instruct-v0.1-offloading-demo --quiet --local-dir Mixtral-8x7B-Instruct-v0.1-offloading-demo

# clear_output()

/bin/bash: line 1: warning: setlocale: LC_ALL: cannot change locale (en_US.UTF-8): No such file or directory
/bin/bash: line 1: ldconfig: command not found


In [11]:
import sys
import os

# Get the absolute path to the project root directory
project_root = os.path.abspath(os.path.join(os.getcwd(), '..'))
print(f"Project root: {project_root}")

# Add the project root to Python path
sys.path.insert(0, project_root)

# Now import other dependencies
import torch
import numpy
from torch.nn import functional as F
from hqq.core.quantize import BaseQuantizeConfig
from huggingface_hub import snapshot_download
from IPython.display import clear_output
from tqdm.auto import trange
from transformers import AutoConfig, AutoTokenizer
from transformers.utils import logging as hf_logging

# Import project specific modules
from src.build_model import OffloadConfig, QuantConfig, build_model

Project root: /home/ljw/mixtral-offloading


## Clean up current memory usage

In [10]:
import torch
import gc

# 1. 创建 globals() 的副本进行迭代，以避免迭代时修改字典的错误
all_globals = list(globals().items())

# 2. 删除模型和其他大型变量
if 'model' in globals():
    del model

# 3. 遍历所有全局变量，删除任何可能占用显存的 torch.Tensor 对象
tensors_to_del = []
for var_name, var_value in all_globals:
    if isinstance(var_value, torch.Tensor):
        tensors_to_del.append(var_name)

for var_name in tensors_to_del:
    if var_name in globals():
        del globals()[var_name]

# 4. 强制执行垃圾回收
gc.collect()

# 5. 清理 CUDA 缓存
if torch.cuda.is_available():
    torch.cuda.empty_cache()
    print("CUDA cache emptied.")

print("Memory and GPU memory cleanup complete.")

CUDA cache emptied.
Memory and GPU memory cleanup complete.


## Initialize model

In [12]:
# Initialize model parameters
model_name = "mistralai/Mixtral-8x7B-Instruct-v0.1"
quantized_model_name = "lavawolfiee/Mixtral-8x7B-Instruct-v0.1-offloading-demo"
state_path = "Mixtral-8x7B-Instruct-v0.1-offloading-demo"

config = AutoConfig.from_pretrained(quantized_model_name)

device = torch.device("cuda:0")

##### Change this to 5 if you have only 12 GB of GPU VRAM #####
# offload_per_layer = 4
offload_per_layer = 5
###############################################################

num_experts = config.num_local_experts

##### Change Cache Strategy as you want #####
cache_strategy = "lfu" #Options: "lru", "lfu", "random"

offload_config = OffloadConfig(
    main_size=config.num_hidden_layers * (num_experts - offload_per_layer),
    offload_size=config.num_hidden_layers * offload_per_layer,
    buffer_size=4,
    offload_per_layer=offload_per_layer,
    cache_strategy=cache_strategy,
)


attn_config = BaseQuantizeConfig(
    nbits=4,
    group_size=64,
    quant_zero=True,
    quant_scale=True,
)
attn_config["scale_quant_params"]["group_size"] = 256


ffn_config = BaseQuantizeConfig(
    nbits=2,
    group_size=16,
    quant_zero=True,
    quant_scale=True,
)
quant_config = QuantConfig(ffn_config=ffn_config, attn_config=attn_config)


model = build_model(
    device=device,
    quant_config=quant_config,
    offload_config=offload_config,
    state_path=state_path,
)

AcceleratorError: CUDA error: unspecified launch failure
CUDA kernel errors might be asynchronously reported at some other API call, so the stacktrace below might be incorrect.
For debugging consider passing CUDA_LAUNCH_BLOCKING=1
Compile with `TORCH_USE_CUDA_DSA` to enable device-side assertions.


## Run the model

In [9]:
from transformers import AutoTokenizer, TextStreamer

tokenizer = AutoTokenizer.from_pretrained(model_name)
streamer = TextStreamer(tokenizer, skip_prompt=True, skip_special_tokens=True)

chat_history = []  # 用于保存所有历史对话

while True:
    print("User: ", end="")
    user_input = input()
    print(f"{user_input}\n")

    # 添加用户输入到历史
    chat_history.append(dict(role="user", content=user_input))

    # 用完整历史拼接 input_ids
    input_ids = tokenizer.apply_chat_template(chat_history, return_tensors="pt").to(device)
    attention_mask = torch.ones_like(input_ids)

    print("Mixtral: ", end="")
    result = model.generate(
        input_ids=input_ids,
        attention_mask=attention_mask,
        streamer=streamer,  # streamer会自动打印输出
        do_sample=True,
        temperature=0.9,
        top_p=0.9,
        max_new_tokens=512,
        pad_token_id=tokenizer.eos_token_id,
        return_dict_in_generate=True,
        output_hidden_states=False,
    )
    print("\n")

    # 解码新回复用于保存到历史记录
    output_ids = result["sequences"][0]
    reply = tokenizer.decode(output_ids[input_ids.shape[1]:], skip_special_tokens=True)
    
    # 添加机器人回复到历史
    chat_history.append(dict(role="assistant", content=reply))

User: what is ong choy

Mixtral: Ong Choy, also known as Chinese Choi Sum or Yellow Chives, is a type of leafy green vegetable that is commonly consumed in Asian cuisines, especially in Chinese cuisine. It is a member of the Brassica family, which also includes 

KeyboardInterrupt: 